# K Nearest Neighbors - Classification

K nearest neighbors is a simple algorithm that stores all available cases and classifies new cases based on a similarity measure (e.g., distance functions). KNN has been used in statistical estimation and pattern recognition already in the beginning of 1970’s as a non-parametric technique. 

A case is classified by a majority vote of its neighbors, with the case being assigned to the class most common amongst its K nearest neighbors measured by a distance function. If K = 1, then the case is simply assigned to the class of its nearest neighbor. 

<img src='../../img/KNN_similarity.png'>

It should also be noted that all three distance measures are only valid for continuous variables. In the instance of categorical variables the **Hamming distance** must be used. It also brings up the issue of standardization of the numerical variables between 0 and 1 when there is a mixture of numerical and categorical variables in the dataset.

<img src='../../img/KNN_hamming.png'>

For example, Consider the following data concerning credit default. Age and Loan are two numerical variables (predictors) and Default is the target.

<img src='../../img/KNN_example_1.png' height="600" width="800">

We can now use the training set to classify an unknown case (Age=48 and Loan=$142,000) using Euclidean distance. If K=1 then the nearest neighbor is the last case in the training set with Default=Y.

$ D = Sqrt[(48-33)^2 + (142000-150000)^2] = 8000.01  >> Default=Y $

<img src='../../img/KNN_example_2.png' height="600" width="800">

### Code Dictionary
code | description
-----|------------
`KNeighborsClassifier()` | K Nearest Neighbor Classification.
`.meshgrid` |  Generate a matrix of the the specified ranges.
`confusion_matrix` |  Describe the confusion matrix displaying the ratio of the actual and prediction models output.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
dataset = pd.read_csv('Social_Network_Ads.csv')

In [ ]:
dataset.head()

In [ ]:
X = dataset[['Age','EstimatedSalary']].values
y = dataset['Purchased'].values

### We are interested to predict whether if someone buy through the ad

In [ ]:
dataset.groupby('Purchased').count()['User ID']

Following the Supervised Learning steps

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

#### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

We invoke the classifier model

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5, metric='minkowski')

In [ ]:
classifier.fit(X_train, y_train)

In [ ]:
y_pred_class = classifier.predict(X_test)

In [ ]:
y_pred_class

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test, y_pred_class)

In [ ]:
from matplotlib.colors import ListedColormap
X_set, y_set = X_test, y_test
X1, X2 = np.meshgrid(np.arange(start = X_set[:, 0].min() - 1, stop = X_set[:, 0].max() + 1, step = 0.01),
                     np.arange(start = X_set[:, 1].min() - 1, stop = X_set[:, 1].max() + 1, step = 0.01))
plt.figure(figsize=(12, 8))
plt.contourf(X1, X2, classifier.predict(np.array([X1.ravel(), X2.ravel()]).T).reshape(X1.shape),
             alpha = 0.75, cmap = ListedColormap(('red', 'green')))
plt.xlim(X1.min(), X1.max())
plt.ylim(X2.min(), X2.max())
for i, j in enumerate(np.unique(y_set)):
    plt.scatter(X_set[y_set == j, 0], X_set[y_set == j, 1],
                c = ListedColormap(('red', 'green'))(i), label = j)
plt.title('Classifier (Test set)')
plt.xlabel('Age')
plt.ylabel('Estimated Salary')
plt.legend()
plt.show()

## Classification Metrics

In [ ]:
from sklearn.datasets import make_classification
from sklearn.metrics import roc_curve, auc, accuracy_score

### Accuracy Score

![Accuracy](../../img/acc.gif)

This works well only if there are equal number of samples belonging to each class.

In [ ]:
accuracy_score(y_test, y_pred_class)

### Confusion Matrix
A confusion matrix is a table that is often used to describe the performance of a classification model (or "classifier") on a set of test data for which the true values are known.

<img style="float:left;" src="https://image.slidesharecdn.com/finalcustlingprofiling-160226163538/95/customer-linguistic-profiling-10-638.jpg?cb=1456504658" width=600 height=300>

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, y_pred_class))

The confusion matrix enables us to derive other important classification metrics:

- *Precision* : It is the number of correct positive results divided by the number of positive results predicted by the classifier.
- *Recall* : It is the number of correct positive results divided by the number of all relevant samples (all samples that should have been identified as positive).
- *F1-Score* : It is the Harmonic Mean between precision and recall. It tries to find the balance between precision and recall.

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_class))

### Area Under Curve
Area under ROC Curve (or AUC for short) is a performance metric for binary classification problems.

The AUC represents a model’s ability to discriminate between positive and negative classes. An area of 1.0 represents a model that made all predictions perfectly. An area of 0.5 represents a model as good as random.

True Positive Rate (Sensitivity) :  True Positive Rate corresponds to the proportion of positive data points that are correctly considered as positive, with respect to all positive data points.

![tpr](../../img/tpr.gif)

False Positive Rate (Specificity) : False Positive Rate corresponds to the proportion of negative data points that are mistakenly considered as positive, with respect to all negative data points.

![fpr](../../img/fpr.gif)

#### Let's create a template

In [ ]:
#from sklearn.svm import SVC
#X, y = make_classification(n_samples=1000)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
#clf = SVC(probability=True).fit(X_train, y_train)
#y_pred = clf.predict_proba(X_test)

In [ ]:
y_test

In [ ]:
def plot_roc(y_test, clf):
    y_pred = clf.predict_proba(X_test)[:, 1]
    fpr, tpr, ths = roc_curve(y_test, y_pred)
    auc_val = auc(fpr, tpr)
    ax1.plot(fpr, tpr, color="red", label="AUC = {0:.2f}".format(auc_val))
    ax1.set_xlabel("False Positive Rate")
    ax1.set_ylabel("True Positive Rate")
    plt.title("ROC Curve\n")
    ax1.set_xlim((-0.01, 1.01))
    ax1.set_ylim((-0.01, 1.01))
    ax1.set_xticks(np.linspace(0, 1, 11))
    ax1.plot([0, 1], [0, 1], color="blue")
    ax1.legend(loc="lower right")

In [ ]:
fig = plt.figure(figsize=(12, 8))
ax1 = fig.add_subplot(111)
plot_roc(y_test, classifier)#clf)

In [ ]:
y_test

# Model Evaluation

- Goal has been to predict the **response value** of an **unknown observation**
    - predict the species of an unknown iris
- Made predictions using KNN models with **different values of K**
- Need a way to choose the **"best" model**: the one that "generalizes" to "out-of-sample" data

**Solution:** Create a procedure that **estimates** how well a model is likely to perform on out-of-sample data and use that to choose between models.

**Note:** These procedures can be used with **any machine learning model**, not only KNN.

### Using the train/test split procedure (K=1)

In [ ]:
#train the model on the training set (using K=1)
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train)

#### Increasing the neighbors

In [ ]:
classifier50 = KNeighborsClassifier(n_neighbors=50)
classifier50.fit(X_train, y_train)
print (accuracy_score(y_test, classifier50.predict(X_test)))

#### Searching for the "best" value of K

Calculate TRAINING ERROR and TESTING ERROR for K=1 through 100

In [ ]:
k_range = range(1, 101)
training_error = []
testing_error = []

for k in k_range:
    # instantiate the model with the current K value
    knn = KNeighborsClassifier(n_neighbors=k)

    # calculate training error
    knn.fit(X, y)
    y_pred_class = knn.predict(X)
    training_accuracy = accuracy_score(y, y_pred_class)
    training_error.append(1 - training_accuracy)
    
    # calculate testing error
    knn.fit(X_train, y_train)
    y_pred_class = knn.predict(X_test)
    testing_accuracy = accuracy_score(y_test, y_pred_class)
    testing_error.append(1 - testing_accuracy)

Creating a DataFrame of K, training error, and testing error

In [ ]:
df = pd.DataFrame({'K': k_range,
                   'training error':training_error,
                   'testing error':testing_error}).set_index('K').sort_index(ascending=False)

In [ ]:
df.head()

Plotting the relationship between K (HIGH TO LOW) and TESTING ERROR

### Training error versus testing error

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(df.index, df['testing error'], color='g')
plt.plot(df.index, df['training error'])
plt.xlabel('Value of K for KNN')
plt.ylabel('Error (lower is better)')

- **Training error** decreases as model complexity increases (lower value of K)
- **Testing error** is minimized at the optimum model complexity

In [ ]:
# find the minimum testing error and the associated K value
df.sort_values(by=['testing error']).head()

What could we conclude?

![Bias-variance tradeoff](../../img/training_testing_error.png)